In [1]:
import numpy as np
from sklearn import datasets, linear_model

# Load the diabetes dataset
diabetes_X, diabetes_y = datasets.load_diabetes(return_X_y=True)

# Use only one feature
diabetes_X = diabetes_X[:, np.newaxis, 2]

# Split the data into training/testing sets
diabetes_X_train = diabetes_X[:-20]
diabetes_X_test = diabetes_X[-20:]

# Split the targets into training/testing sets
diabetes_y_train = diabetes_y[:-20]
diabetes_y_test = diabetes_y[-20:]

In [7]:
import mlflow
import rikai
from example import spark

mlflow_tracking_uri = "sqlite:///mlruns.db"
mlflow.set_tracking_uri(mlflow_tracking_uri)

model = linear_model.LinearRegression()
model.fit(diabetes_X_train, diabetes_y_train)

spark.conf.set("rikai.sql.ml.registry.mlflow.tracking_uri", mlflow_tracking_uri)

:: loading settings :: url = jar:file:/Users/da/.pyenv/versions/3.8.10/envs/rikai-example/lib/python3.8/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/da/.ivy2/cache
The jars for the packages stored in: /Users/da/.ivy2/jars
ai.eto#rikai_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ad187b6d-01d1-409a-aec4-5f924e2f33db;1.0
	confs: [default]
	found ai.eto#rikai_2.12;0.0.11 in central
	found org.antlr#antlr4-runtime;4.8-1 in local-m2-cache
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.apache.logging.log4j#log4j-api-scala_2.12;12.0 in central
	found org.scala-lang#scala-reflect;2.12.10 in spark-list
	found org.apache.logging.log4j#log4j-api;2.13.2 in central
	found io.circe#circe-core_2.12;0.12.3 in central
	found io.circe#circe-numbers_2.12;0.12.3 in central
	found org.typelevel#cats-core_2.12;2.0.0 in central
	found org.typelevel#cats-macros_2.12;2.0.0 in central
	found org.typelevel#cats-kernel_2.12;2.0.0 in central
	found io.circe#circe-generic_2.12;0.12.3 in central
	found com.chuusai#shapeless_2.12;2.3.3 in spark-list
	found org.typ

In [32]:
with mlflow.start_run() as run:
    rikai.mlflow.sklearn.log_model(
        model,
        schema="float",
        artifact_path = "model",
        registered_model_name = "diabetes_rikai")

Registered model 'diabetes_rikai' already exists. Creating a new version of this model...
2021/10/14 11:46:45 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: diabetes_rikai, version 3
Created version '3' of model 'diabetes_rikai'.


In [33]:
spark.sql(f"""
CREATE OR REPLACE MODEL diabetes_m USING 'mlflow:///diabetes_rikai'
""")

2021-10-14 11:46:47,461 INFO Rikai (mlflow_registry.py:223): Resolving model diabetes_m from mlflow:///diabetes_rikai
2021-10-14 11:46:47,489 INFO Rikai (base.py:207): Created model inference pandas_udf with name diabetes_m_ffb7cfb7


DataFrame[]

In [34]:
spark.sql(f"""
SELECT ML_PREDICT(diabetes_m, array({diabetes_X_test[0][0]}));
""").show()

+---------------------------------------------+
|diabetes_m_ffb7cfb7(array(0.077863387626902))|
+---------------------------------------------+
|                                    225.97324|
+---------------------------------------------+

